In [699]:
import psycopg2
import pandas as pd
import numpy as np

In [700]:

conn = psycopg2.connect("host=127.0.0.1 dbname=vice user=vice password=vice")
cur = conn.cursor()

In [701]:
#conn = psycopg2.connect(host="127.0.0.1", port="5432", dbname="vice", user="vice", password="vice")

#conn = psycopg2.connect(host="postgres", port="5432", dbname="vice", user="vice", password="vice")

In [728]:
cur.execute("""
SELECT j.id as job_id, 
j.name as job_name, 
j.expected_revenue,
re.id as revenue_id,
re.order_id,
re.order_name,
extract(MONTH from re.month_of_service) as month_of_service,
re.delivered_impressions,
re.revenue
from revenue_entries as re 
left join jobs as j on re.job_id = j.id
where job_id = %s""", '3')

#print(cur.fetchall())

In [729]:
tuples

[(2,
  'Fixture J2',
  Decimal('40.00'),
  4,
  2,
  'Fixture O2',
  1.0,
  1000,
  Decimal('0.00'))]

In [730]:
for row in tuples:
    print("job_id =", row[0])
    print("job_name =", row[1])
    print("expected_revenue =", row[2])
    print("revenue_id =", row[3])
    print("order_id =", row[4])
    print("order_name =", row[5])

    
    print("month_of_service =", row[6])
    print("delivered_impressions =", row[7])
    print("revenue =", row[8], "\n")


job_id = 2
job_name = Fixture J2
expected_revenue = 40.00
revenue_id = 4
order_id = 2
order_name = Fixture O2
month_of_service = 1.0
delivered_impressions = 1000
revenue = 0.00 



In [731]:
columns = ["job_id", "job_name", "expected_revenue", "revenue_id", "order_id",
              "order_name", "month_of_service", "delivered_impressions", "revenue"]
tuples = cur.fetchall()
df = pd.DataFrame(tuples, columns=columns)
df

,job_id,job_name,expected_revenue,revenue_id,order_id,order_name,month_of_service,delivered_impressions,revenue
0,3,Fixture J3,50.00,5,3,Fixture O3,1.0,1000,10.00
1,3,Fixture J3,50.00,6,3,Fixture O3,1.0,1000,10.00
2,3,Fixture J3,50.00,7,3,Fixture O3,1.0,1000,10.00
3,3,Fixture J3,50.00,8,4,Fixture O4,1.0,90000,1005.60
4,3,Fixture J3,50.00,9,4,Fixture O4,2.0,6500,22.78
5,3,Fixture J3,50.00,10,4,Fixture O4,2.0,500,0.00
6,3,Fixture J3,50.00,11,4,Fixture O4,3.0,0,10.00


In [732]:
df['revenue']=df['revenue'].astype(float)
df['expected_revenue']=df['expected_revenue'].astype(float)

In [733]:
#df = df[df['revenue'] > 0]

#df = df[df['delivered_impressions']>0]

In [734]:
df.groupby("order_id")[['delivered_impressions', "revenue"]].sum()

,delivered_impressions,revenue
order_id,,
3,3000,30.00
4,97000,1038.38


In [735]:
print((1038.28)/97000*1000)

10.703917525773196


In [736]:
df["revenue"]/(df['delivered_impressions']/1000)

0    10.000000
1    10.000000
2    10.000000
3    11.173333
4     3.504615
5     0.000000
6          inf
dtype: float64

In [737]:
df['ecpm'] = (df['revenue'] / df['delivered_impressions'] )*1000

In [738]:
overall_df = df.groupby(['job_name', 'job_id']).agg({"expected_revenue": "mean", "revenue": "sum", \
                           "month_of_service":pd.Series.nunique}).reset_index()

In [739]:
overall_df['met_expected_revenue'] = overall_df.apply(lambda x: True if (x['expected_revenue'] <= x["revenue"])\
                                                      else False, axis =1)

In [740]:
overall_df.head()

,job_name,job_id,expected_revenue,revenue,month_of_service,met_expected_revenue
0,Fixture J3,3,50.0,1068.38,3.0,True


In [741]:
dict_results = {"job_name": overall_df["job_name"].values[0]}
dict_results["expected_revenue"] = overall_df["expected_revenue"].values[0]
dict_results["total_revenue"] = round(overall_df["revenue"].values[0], 2)
dict_results["revenue_target_met"] = overall_df["met_expected_revenue"][0]
dict_results["month_of_service_count"] = int(round(overall_df["month_of_service"].values[0]))

In [742]:
df.replace(np.inf, 0, inplace = True)

In [743]:
order_df = df.groupby(["order_id", "order_name"]).agg({"revenue": "sum", "ecpm": "mean", \
                                                     "delivered_impressions": "sum"}).reset_index()

In [744]:
order_df.head()

,order_id,order_name,revenue,ecpm,delivered_impressions
0,3,Fixture O3,30.00,10.000000,3000
1,4,Fixture O4,1038.38,3.669487,97000


In [745]:
round(1038.38/(97000/1000),2)

10.7

In [746]:
order_df['total_ecpm'] = order_df['revenue']/(order_df['delivered_impressions']/1000)

In [747]:
order_df

,order_id,order_name,revenue,ecpm,delivered_impressions,total_ecpm
0,3,Fixture O3,30.00,10.000000,3000,10.000000
1,4,Fixture O4,1038.38,3.669487,97000,10.704948


In [748]:
dict_orders = {}
for row in order_df.iterrows():
    temp_dict = {}
    temp_dict["order_name"] = row[1]["order_name"]
    temp_dict["total_revenue"] = round(row[1]["revenue"],2)
    temp_dict["average_ecpm"] = round(row[1]["ecpm"],2)
    temp_dict["total_ecpm"] = round(row[1]["total_ecpm"],1)
    dict_orders[row[1]["order_id"]] = temp_dict
    #dict_orders[row[1]["order_id"] = temp_dict

{'order_name': 'Fixture O3', 'total_revenue': 30.0, 'average_ecpm': 10.0, 'total_ecpm': 10.0}
{'order_name': 'Fixture O4', 'total_revenue': 1038.38, 'average_ecpm': 3.67, 'total_ecpm': 10.7}


In [749]:
for row in order_df.iterrows():
    print(row[1]["order_name"])

Fixture O3
Fixture O4


In [750]:
dict_results["orders"] = dict_orders

In [751]:
dict_results

{'job_name': 'Fixture J3',
 'expected_revenue': 50.0,
 'total_revenue': 1068.38,
 'revenue_target_met': True,
 'month_of_service_count': 3,
 'orders': {3: {'order_name': 'Fixture O3',
   'total_revenue': 30.0,
   'average_ecpm': 10.0,
   'total_ecpm': 10.0},
  4: {'order_name': 'Fixture O4',
   'total_revenue': 1038.38,
   'average_ecpm': 3.67,
   'total_ecpm': 10.7}}}

In [752]:
tester = {
            "job_name": "Fixture J3",
            "expected_revenue": 50.00,
            "total_revenue": 1068.38,
            "revenue_target_met": True,
            "month_of_service_count": 3,
            "orders": {
                3: {
                    "order_name": "Fixture O3",
                    "total_revenue": 30.00,
                    "average_ecpm": 10.00,
                    "total_ecpm": 10.00
                },
                4: {
                    "order_name": "Fixture O4",
                    "total_revenue": 1038.38,
                    "average_ecpm": 3.67,
                    "total_ecpm": 10.70
                }
            }
        }

In [753]:
print(dict_results == tester)

True
